In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


/var/folders/25/82ztpk21087b258r7gs0n3m00000gn/T/ipykernel_18125/2259232837.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


We will use the "living planet index" for this dataset. Quoting directly from https://www.livingplanetindex.org/ : 
"The Living Planet Index (LPI) is a measure of the state of the world's biological diversity based on population trends of vertebrate species from terrestrial, freshwater and marine habitats. The LPI was adopted by the Convention of Biological Diversity (CBD) as an indicator of progress towards its 2011-2020 targets and can play an important role in monitoring progress towards the post-2020 goals and targets negotiated at COP15 this December."

We will try to predict the LPI based on co2 emissions, world population and world gdp.

In [2]:
# min and max year for the LPI in my dataset
min_year = 1970
max_year = 2018

In [3]:
df_co2 = pd.read_csv('time_series_data/annual-co2-emissions-per-country.csv')
df_co2 = df_co2[df_co2['Year'] >= min_year]
df_co2 = df_co2[df_co2['Year'] <= max_year]
df_co2_by_year = df_co2[df_co2['Entity'] == 'World']
df_co2_by_year = pd.DataFrame({
    'Year': df_co2_by_year['Year'],
    'co2': df_co2_by_year['Annual CO₂ emissions'],
})
df_full = df_co2_by_year
df_full

,Year,co2
29989,1970,1.489816e+10
29990,1971,1.550279e+10
29991,1972,1.622344e+10
29992,1973,1.708350e+10
29993,1974,1.700938e+10
29994,1975,1.704998e+10
29995,1976,1.798524e+10
29996,1977,1.849465e+10
29997,1978,1.906396e+10
29998,1979,1.960300e+10


In [4]:
df_population = pd.read_csv('time_series_data/population.csv')
df_population = df_population[df_population['Year'] >= min_year]
df_population = df_population[df_population['Year'] <= max_year]
df_population_by_year = df_population[df_population['Entity'] == 'World']
df_population_by_year = pd.DataFrame({
    'Year': df_population_by_year['Year'],
    'population': df_population_by_year['Population (historical estimates)'],
})
df_full = pd.merge(df_full, df_population_by_year, on='Year')
df_full

,Year,co2,population
0,1970,1.489816e+10,3695390208
1,1971,1.550279e+10,3770163200
2,1972,1.622344e+10,3844800768
3,1973,1.708350e+10,3920251392
4,1974,1.700938e+10,3995516928
5,1975,1.704998e+10,4069437184
6,1976,1.798524e+10,4142505728
7,1977,1.849465e+10,4215772416
8,1978,1.906396e+10,4289657600
9,1979,1.960300e+10,4365582848


In [5]:
df_gdp_growth = pd.read_csv('time_series_data/yearly_gdp_growth.csv')
# assume no value for gdp represents 0
df_gdp_growth.fillna(0, inplace=True)
global_gdps = []
for year in range(min_year, max_year+1):
    global_gdps.append(df_gdp_growth[str(year)].mean())

df_full['gdp_growth'] = global_gdps
df_full

,Year,co2,population,gdp_growth
0,1970,1.489816e+10,3695390208,3.981530
1,1971,1.550279e+10,3770163200,3.116490
2,1972,1.622344e+10,3844800768,3.164753
3,1973,1.708350e+10,3920251392,2.977396
4,1974,1.700938e+10,3995516928,3.157240
5,1975,1.704998e+10,4069437184,1.693890
6,1976,1.798524e+10,4142505728,3.785169
7,1977,1.849465e+10,4215772416,2.854632
8,1978,1.906396e+10,4289657600,2.739368
9,1979,1.960300e+10,4365582848,2.671118


In [6]:
df_lpi = pd.read_csv('time_series_data/global-living-planet-index.csv')
df_lpi = df_lpi[df_lpi['Entity'] == 'World']

lpis = list(df_lpi['Living Planet Index'])
lpis

[100.0,
 99.100274,
 97.929883,
 96.78284,
 95.34543,
 93.5558,
 91.48922,
 88.535243,
 85.68405,
 83.34747999999999,
 81.508094,
 79.83687499999999,
 77.281386,
 74.518186,
 71.832025,
 69.753116,
 68.125665,
 66.32249,
 64.5467,
 63.574344,
 62.68112,
 61.60914,
 60.11985,
 58.109295,
 56.2009,
 54.0361,
 53.04214,
 51.750076,
 50.466454000000006,
 48.66438,
 47.221538,
 45.67137,
 44.498393,
 43.515655,
 42.92783,
 42.203152,
 41.295153,
 40.142804,
 38.719392,
 36.877126,
 35.15315,
 33.44104,
 32.37109,
 31.738263,
 31.66902,
 31.627533000000003,
 31.14541,
 30.88527,
 30.897567]

In [7]:
df_full['lpi'] = lpis
df_full
df_full.to_csv('time_series_data/LPI_data.csv')